Author: Pratham Gujar

Email: pratham.gujar30@gmail.com

# Imports and Installations

In [1]:
%pip install qiskit -q
%pip install pylatexenc -q
%pip install qiskit_machine_learning -q
%pip install imblearn -q
%pip install qiskit-algorithms -q
%pip install qiskit-aer -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 68.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.6/231.6 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.5/310.5 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 64.8 MB/s eta 0:00:00


In [2]:
# Imports
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import pandas as pd
import numpy as np
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.circuit.library import ZZFeatureMap
from qiskit_aer import AerSimulator
from qiskit_machine_learning.algorithms import QSVC
from sklearn.metrics import classification_report, recall_score
import itertools
from time import time

# Part 1: Simple AQSVC For German Credit Data

I used a processed version of the Statlog German Credit data where the features are better categorized for the QSVC model to use. The alphanumeric codes were removed and replaced with extra columns in the dataset.


In [3]:
def adjusted_features(df, save=True, output_file='German_Adjusted_Features.csv'):
  """
  Adjusts the features based on correlation thresholds.

  Args:
      df (pandas.DataFrame): The input DataFrame.
      save (bool, optional): Whether to save the adjusted features. Defaults to True.
      output_file (str, optional): The output file path. Defaults to 'German_Adjusted_Features.csv'.

  Returns:
      pandas.DataFrame: The DataFrame with adjusted features.
  """

  correlation_matrix = df.corr()

  # Extract correlations with the target 'classification'
  correlations_with_target = correlation_matrix['classification'].drop('classification').sort_values(ascending=False)

  adjusted_features = correlations_with_target[
      (correlations_with_target >= 0.55) | (correlations_with_target <= -0.05)
  ].index.tolist()


  adjusted_features_data = df[adjusted_features + ['classification']]

  if save:
    adjusted_features_data.to_csv(output_file, index=False)

  return adjusted_features_data



I noticed that the QSVC model had trouble fitting the minority data properly since there were much more features with a positive correlation than a negative correlation. To better the fitting of the minority class, I reduced the features from the dataset which were positively correlated to give more focus to the minority features. I made sure to not trim too much so the score remained above at least 0.7.

In [4]:
def preprocess(X, y, test_size=0.2, random_state=42, n_dim=4):
  """
  Preprocesses data using PCA based on the number of qubits (n_dim)

  Args:
      X (numpy.ndarray): Input features.
      y (numpy.ndarray): Target labels.
      test_size (float, optional): Proportion of the dataset to include in the test split. Defaults to 0.2.
      random_state (int, optional): Seed for the random number generator. Defaults to 42.
      n_dim (int, optional): Number of dimensions for the PCA. Defaults to 4.

  Returns:
      tuple: A tuple containing the preprocessed training data, test data, training labels, and test labels.
  """

  sample_train, sample_test, label_train, label_test = train_test_split(X, y, test_size=test_size, random_state=random_state)


  # Reduce dimensions
  pca = PCA(n_components=n_dim).fit(sample_train)
  sample_train = pca.transform(sample_train)
  sample_test = pca.transform(sample_test)

  # Normalize
  std_scale = StandardScaler().fit(sample_train)
  sample_train = std_scale.transform(sample_train)
  sample_test = std_scale.transform(sample_test)

  # Scale
  samples = np.append(sample_train, sample_test, axis=0)
  minmax_scaler = MinMaxScaler((-1,1)).fit(samples)
  sample_train = minmax_scaler.transform(sample_train)
  sample_test = minmax_scaler.transform(sample_test)

  # Select
  train_size = 100
  sample_train = sample_train[:train_size]
  label_train = label_train[:train_size].values

  test_size = 20
  sample_test = sample_test[:test_size]
  label_test = label_test[:test_size].values

  return sample_train, sample_test, label_train, label_test

In [5]:
def run_qsvc(sample_train, label_train, sample_test, n_dim=4, save=True, output_model='german_qsvc.joblib', entanglement='linear', reps=2):
  """
  Runs QSVC on the provided data.

  Args:
      sample_train (numpy.ndarray): Training data.
      label_train (numpy.ndarray): Training labels.
      sample_test (numpy.ndarray): Test data.
      n_dim (int, optional): Number of dimensions for the QSVC. Defaults to 4.
      save (bool, optional): Whether to save the trained QSVC model. Defaults to True.

  Returns:
      numpy.ndarray: Predictions for the test data.
  """

  zz_map = ZZFeatureMap(feature_dimension=n_dim, reps=reps, entanglement=entanglement, insert_barriers=True)

  zz_kernel = FidelityQuantumKernel(feature_map=zz_map)

  qsvc = QSVC(quantum_kernel=zz_kernel)
  qsvc.fit(sample_train, label_train)

  if save:
    qsvc.save(output_model)

  predictions = qsvc.predict(sample_test)

  return predictions

In [6]:
def optimize_qsvc(X, y, test_params=None, n=10, seed=42, verbose=True):
    """
    Optimization function to find the best repetitions (reps), entanglement, and number of qubits (n_dim)
    for the QSVC model.

    Args:
        X (numpy.ndarray): Input features.
        y (numpy.ndarray): Target labels.

    Returns:
        tuple: A tuple containing the best minority recall, the best parameters

    """
    best_recall = 0
    best_params = {}
    best_report = ""
    if test_params is None:
      reps_range = [1, 2, 3]
      entanglement_options = ['linear', 'full', 'circular', 'sca']
      n_dim_range = [2, 4, 6]
    else:
      reps_range = test_params['reps']
      entanglement_options = test_params['entanglement']
      n_dim_range = test_params['n_dim']

    params_combinations = list(itertools.product(reps_range, entanglement_options, n_dim_range))[:n]
    np.random.shuffle(params_combinations)

    for i, (reps, entanglement, n_dim) in enumerate(params_combinations):
        start = time()

        sample_train, sample_test, label_train, label_test = preprocess(X, y, n_dim=n_dim, random_state=seed)

        predictions = run_qsvc(sample_train, label_train, sample_test, n_dim=n_dim, reps=reps, entanglement=entanglement, save=False)

        minority_recall = recall_score(label_test, predictions, pos_label=-1)

        params = {
            'reps': reps,
            'entanglement': entanglement,
            'n_dim': n_dim
        }

        if minority_recall > best_recall:
            best_recall = minority_recall
            best_params = params
            # best_report = classification_report(label_test, predictions)
        if verbose:
          print(f"Iteration {i + 1}: Current Recall: {minority_recall:.2f}, Params: {params}, Time Took: {time()-start:.2f}s")

    return best_recall, best_params

In [8]:
def main():
  df = pd.read_csv('German_Preprocessed.csv')
  df = adjusted_features(df, save=False)

  X = df.drop('classification', axis=1)
  y = df['classification']

  best_recall, best_params = optimize_qsvc(X, y, n=36, seed=42)

  print("\nFinal Results:")
  print("Best Minority Recall:", best_recall)
  print("Best Parameters:", best_params)

main()

Iteration 1: Current Recall: 0.00, Params: {'reps': 1, 'entanglement': 'sca', 'n_dim': 6}, Time Took: 58.63s
Iteration 2: Current Recall: 0.17, Params: {'reps': 3, 'entanglement': 'linear', 'n_dim': 2}, Time Took: 29.37s
Iteration 3: Current Recall: 0.00, Params: {'reps': 2, 'entanglement': 'circular', 'n_dim': 6}, Time Took: 77.14s
Iteration 4: Current Recall: 0.17, Params: {'reps': 3, 'entanglement': 'full', 'n_dim': 2}, Time Took: 29.56s
Iteration 5: Current Recall: 0.00, Params: {'reps': 2, 'entanglement': 'sca', 'n_dim': 6}, Time Took: 74.83s
Iteration 6: Current Recall: 0.00, Params: {'reps': 1, 'entanglement': 'full', 'n_dim': 6}, Time Took: 72.27s
Iteration 7: Current Recall: 0.33, Params: {'reps': 1, 'entanglement': 'full', 'n_dim': 2}, Time Took: 15.15s
Iteration 8: Current Recall: 0.17, Params: {'reps': 2, 'entanglement': 'circular', 'n_dim': 4}, Time Took: 50.19s
Iteration 9: Current Recall: 0.00, Params: {'reps': 3, 'entanglement': 'linear', 'n_dim': 4}, Time Took: 59.89s


# Part 2: Ways to improve

The current code utilizes the **Aer Simulator** rather than an actual quantum backend. This limits the number of qubits we can use to create a feature map for the data. There could be more accurate models that utilize 20 or 30 qubits which would be too long to classically simulate.




Another proposition would be to deeply investigate the dataset and identify possible pairwise interactions amongst features. This would allow us to alter the **ZZFeatureMap** to better map the controlled gates with the particular features. This should make evaluating the kernel matrix more efficient. Furthermore, this could make the model's outcome more accurate but this would depend on several more factors. By doing this, we are reducing the complexity of the circuit which would cause some encodings to no longer happen. Thus, we lose some information which may lead to a more innacurate model. We must also be careful of overfitting to ensure the model can properly predict outcomes of unseen data.

If we do run this on a quantum computer, we will need to consider noise and the fault tolerance of the computer. We will need to use error correction syndromes (such as the Perfect Code algorithm) to identify potential errors and fix them before improperly computing the kernel matrix.

# Report and Results

## Abstract

Using the code above, I found that using 2 repetitions with linear entanglement and 4 qubits yielded the best result of 0.67 minority recall. Different results could be achieved by changing the test/train split but it will not vary by too much.

## Tools Used

For this, I utilized Qiskit for modeling my machine learning implementation. I also used a preprocessed version of the Statlot German Credit data where the alphanumerical values in some columns are restructured into their own indvidual columns with numerical values. For data handling, I used pandas, numpy, and sklearn. Much of the preprocessing utilizes functions from sklearn.

## Strategy

First, I filter the German Credit Data to find features more closely correlated to the minority target value. I tweaked this to increase minority recall and maximize the overall accuracy score for the SVC fitting. This is found in the `adjusted_features()` function.

I preprocess the restructured German Credit Data using PCA to reduce the dimensionality to match the number of qubits. For example, if the data is reduced to 4 dimensions, then there are 4 qubits to represent the features.

The features are then normalized using a Z-score normalization to have a mean of 0 and a standard deviation of 1. The features are then scaled to -1 and 1 to fascilitate machine learning fitting. Finally, a sample of the training and testing data is selected to minimized computational computing.

I chose a `ZZFeatureMap` to map the features to the quantum map. This handles pairwise interactions between qubits which considers interactions between features and how they affect the produced target. I create a quantum kernel using the feature map and the kernel is evaluated when using the `qsvc` function. The evaluated kernel matrix is used in the generated quantum circuit to find the best model to predict test data. Refer to the `run_qsvc()` function to see the implementation.

The `optimize_qsvc()` function wraps around the `run_qsvc()` function to try various quantum hyperparameters such as repetitions `reps`, entanglement `entanglement`, and number of qubits `n_dims`. There are default values assigned if the user does not feed in `test_params`.

## Results

With my strategy, I determined that 2 repetitions of the feature circuit, with a linear entanglement, and 4 qubits yielded the best minority recall of 0.67. The testing was not comprehensive since I was limited by computational power and there could be a better configuration. However, I noticed that as the qubit number deviated from 4, the minority score decreased until reaching 0. Thus, it appears that 4 qubits may already be an optimized hyperparameter for the QSVC solution.

## Conclusion

The code above, found a configuration where the minority recall is above 0.5 without sacrificing much from the accuracy score (0.75). The `ZZFeatureMap` provided best results as it considered interactions between features and mapped the features the best. A custom Pauli Map could be implemented to better map the features but more investigation is required. The results obtained did satisfy the condition for achieving a minority score of 0.5.